Verificação Caminho

In [0]:
dbutils.fs.ls('/Workspace/csv/')

Carga Inicial dos arquivos CSV

In [0]:
# Caminho dos arquivos CSV no Workspace
path_full = '/Workspace/csv/campeonato-brasileiro-full.csv'
path_cartoes = '/Workspace/csv/campeonato-brasileiro-cartoes.csv'
path_gols = '/Workspace/csv/campeonato-brasileiro-gols.csv'
path_estatisticas = '/Workspace/csv/campeonato-brasileiro-estatisticas-full.csv'

# Carregar os arquivos CSV como DataFrames
df_full = spark.read.csv(path_full, header=True, inferSchema=True)
df_cartoes = spark.read.csv(path_cartoes, header=True, inferSchema=True)
df_gols = spark.read.csv(path_gols, header=True, inferSchema=True)
df_estatisticas = spark.read.csv(path_estatisticas, header=True, inferSchema=True)

# Mostrar as primeiras linhas de cada DataFrame
df_full.show(5)
df_cartoes.show(5)
df_gols.show(5)
df_estatisticas.show(5)